#### Twitter Stream

In [2]:
from tweepy import (Stream, OAuthHandler)
from tweepy.streaming import StreamListener
 
class Listener(StreamListener):

    tweet_counter = 0 # Static variable
    
    def login(self):
        CONSUMER_KEY = 'yoIwFkjZGYDa49aO16XqSNqcN'
        CONSUMER_SECRET = 'gl4LQOItV7Z1aFwNrlvaiKJ3t8o8h99blMIAmnmdHxYjzjRAxO'
        ACCESS_TOKEN = '624310916-E7fDF2IE8P6bfY1oVFglASf6F8RnxMd3vgSXFqnZ'
        ACCESS_TOKEN_SECRET = 'ID9JcoXHsDcKtvNcnmBGcCQhUlO0wmwAxBJ6LCesiUAas'

        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth
    
    def on_status(self, status):
        Listener.tweet_counter += 1
        print(str(Listener.tweet_counter) + '. Screen name = "%s" Tweet = "%s"'
              %(status.author.screen_name, status.text.replace('\n', ' ')))

        if Listener.tweet_counter < Listener.stop_at:
            return True
        else:
            print('Max num reached = ' + str(Listener.tweet_counter))
            return False

    def getTweetsByGPS(self, stop_at_number, latitude_start, longitude_start, latitude_finish, longitude_finish):
        try:
            Listener.stop_at = stop_at_number # Create static variable
            auth = self.login()
            streaming_api = Stream(auth, Listener(), timeout=60) # Socket timeout value
            streaming_api.filter(follow=None, locations=[latitude_start, longitude_start, latitude_finish, longitude_finish])
        except KeyboardInterrupt:
            print('Got keyboard interrupt')

    def getTweetsByHashtag(self, stop_at_number, hashtag):
        try:
            Listener.stopAt = stop_at_number
            auth = self.login()
            streaming_api = Stream(auth, Listener(), timeout=60)
            # Atlanta area.
            streaming_api.filter(track=[hashtag])
        except KeyboardInterrupt:
            print('Got keyboard interrupt')

listener = Listener()
listener.getTweetsByGPS(20, -84.395198, 33.746876, -84.385585, 33.841601) # Atlanta area. Tool to find coordinates for any place: http://boundingbox.klokantech.com/ (use CSV as the output format)

1. Screen name = "GahDamnNick" Tweet = "@KaiWasTakenX2 It’s people that’s taking advantage of the protest making the people that ACTUALLY want justice look bad"
2. Screen name = "rileyfetter" Tweet = "https://t.co/hXjb63aeGi"
3. Screen name = "TheTaylorPerrin" Tweet = "@Hok_keis Lmaoooo not “so fuck her” 😂😂"
4. Screen name = "brendafeaster5" Tweet = "@gtconway3d Section 230 needs some changes"
5. Screen name = "ameer_ice_drake" Tweet = "Just posted a photo @ Atlanta, Georgia https://t.co/q8dRByhXLA"
6. Screen name = "Abby_Spunky" Tweet = "Y’all Peachtree was a peachy af 🥰"
7. Screen name = "kailynnlee" Tweet = "This what I need to do I can’t skip into this dark space again"
8. Screen name = "EmbraceMeHere" Tweet = "Someone said Zimmerman is just a poor batman and i don’t see the issue"
9. Screen name = "mxrlonszn" Tweet = "With everything that’s going on I can’t even imagine what QAnon conspiracy theorists are cooking up"
10. Screen name = "QueenNala___" Tweet = "I was just blasting I’

## 3. Cleaning tweets:

In [3]:
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

punctuation += '´΄’…“”–—―»«' # string.punctuation misses these.

cache_english_stopwords = stopwords.words('english') # Could speed up code by making this a set

def tweet_clean(tweet):
    print('Original tweet:', tweet, '\n')
    # Remove HTML special entities (e.g. &amp;)
    tweet_no_special_entities = re.sub(r'\&\w*;', '', tweet)
    print('No special entitites:', tweet_no_special_entities, '\n')
    # Remove tickers (Clickable stock market symbols that work like hashtags and start with dollar signs instead)
    tweet_no_tickers = re.sub(r'\$\w*', '', tweet_no_special_entities) # Substitute. $ needs to be escaped because it means something in regex. \w means alphanumeric char or underscore.
    print('No tickers:', tweet_no_tickers, '\n')
    # Remove hyperlinks
    tweet_no_hyperlinks = re.sub(r'https?:\/\/.*\/\w*', '', tweet_no_tickers)
    print('No hyperlinks:', tweet_no_hyperlinks, '\n')
    # Remove hashtags
    tweet_no_hashtags = re.sub(r'#\w*', '', tweet_no_hyperlinks)
    print('No hashtags:', tweet_no_hashtags, '\n')
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet_no_punctuation = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet_no_hashtags)
    print('No punctuation:', tweet_no_punctuation, '\n')
    # Remove words with 2 or fewer letters (Also takes care of RT)
    tweet_no_small_words = re.sub(r'\b\w{1,2}\b', '', tweet_no_punctuation) # \b represents a word boundary
    print('No small words:', tweet_no_small_words, '\n')
    # Remove whitespace (including new line characters)
    tweet_no_whitespace = re.sub(r'\s\s+', ' ', tweet_no_small_words)
    tweet_no_whitespace = tweet_no_whitespace.lstrip(' ') # Remove single space left on the left
    print('No whitespace:', tweet_no_whitespace, '\n')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet_no_emojis = ''.join(c for c in tweet_no_whitespace if c <= '\uFFFF') # Apart from emojis (plane 1), this also removes historic scripts and mathematical alphanumerics (also plane 1), ideographs (plane 2) and more.
    print('No emojis:', tweet_no_emojis, '\n')
    # Tokenize: Change to lowercase, reduce length and remove handles
    tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True) # reduce_len changes, for example, waaaaaayyyy to waaayyy.
    tw_list = tknzr.tokenize(tweet_no_emojis)
    print('Tweet tokenize:', tw_list, '\n')
    # Remove stopwords
    list_no_stopwords = [i for i in tw_list if i not in cache_english_stopwords]
    print('No stop words:', list_no_stopwords, '\n')
    # Final filtered tweet
    tweet_filtered =' '.join(list_no_stopwords) # ''.join() would join without spaces between words.
    print('Final tweet:', tweet_filtered)

s = '    RT @Amila #Test\nTom\'s newly listed Co. &amp; Mary\'s unlisted     Group to supply tech for nlTK.\nh.. $TSLA $AAPL https:// t.co/x34afsfQsh'
tweet_clean(s)

Original tweet:     RT @Amila #Test
Tom's newly listed Co. &amp; Mary's unlisted     Group to supply tech for nlTK.
h.. $TSLA $AAPL https:// t.co/x34afsfQsh 

No special entitites:     RT @Amila #Test
Tom's newly listed Co.  Mary's unlisted     Group to supply tech for nlTK.
h.. $TSLA $AAPL https:// t.co/x34afsfQsh 

No tickers:     RT @Amila #Test
Tom's newly listed Co.  Mary's unlisted     Group to supply tech for nlTK.
h..   https:// t.co/x34afsfQsh 

No hyperlinks:     RT @Amila #Test
Tom's newly listed Co.  Mary's unlisted     Group to supply tech for nlTK.
h..    

No hashtags:     RT @Amila 
Tom's newly listed Co.  Mary's unlisted     Group to supply tech for nlTK.
h..    

No punctuation:     RT @Amila 
Tom s newly listed Co   Mary s unlisted     Group to supply tech for nlTK 
h     

No small words:      @Amila 
Tom  newly listed    Mary  unlisted     Group  supply tech for nlTK 
     

No whitespace: @Amila Tom newly listed Mary unlisted Group supply tech for nlTK  

No emoji

#### Text Summarization

In [13]:
# Importing the libraries
import bs4 as bs
import urllib.request
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import nltk

# Gettings the data source
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/Global_warming').read()

# Parsing the data/ creating BeautifulSoup object
soup = bs.BeautifulSoup(source,'lxml')

In [14]:
# Fetching the data
text = ""
for paragraph in soup.find_all('p'):
    text += paragraph.text

# Preprocessing the data
text = re.sub(r'\[[0-9]*\]',' ',text)
text = re.sub(r'\s+',' ',text)
clean_text = text.lower()
clean_text = re.sub(r'\W',' ',clean_text)
clean_text = re.sub(r'\d',' ',clean_text)
clean_text = re.sub(r'\s+',' ',clean_text)

In [15]:
# Tokenize sentences
sentences = nltk.sent_tokenize(text)
# Stopword list
stop_words = nltk.corpus.stopwords.words('english')

In [16]:
# Word counts 
word2count = {}
for word in nltk.word_tokenize(clean_text):
    if word not in stop_words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

# Converting counts to weights
max_count = max(word2count.values())
for key in word2count.keys():
    word2count[key] = word2count[key]/max_count

In [17]:
# Product sentence scores    
sent2score = {}
for sentence in sentences:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in word2count.keys():
            if len(sentence.split(' ')) < 25:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]

In [19]:
# Gettings best 5 lines             
import heapq
best_sentences = heapq.nlargest(5, sent2score, key=sent2score.get)

print('---------------------------------------------------------')
for sentence in best_sentences:
    print(sentence)

---------------------------------------------------------
While aerosols typically limit global warming by reflecting sunlight, black carbon in soot that falls on snow or ice can contribute to global warming.
The long-term effects of global warming include further ice melt, ocean warming, sea level rise, and ocean acidification.
Countries work together on climate change under the umbrella of the United Nations Framework Convention on Climate Change (UNFCCC), which has near-universal membership.
Geoengineering or climate engineering is the deliberate large-scale modification of the climate to counteract climate change.
Many arctic animals rely on sea ice, which has been disappearing in a warming Arctic.
